In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import datetime as dt

In [6]:
#Read in the original csv file
AsheCits = pd.read_csv('../data/Asheville_Citations.csv', low_memory=False)

#Data Source: https://data-avl.opendata.arcgis.com/datasets/05af48ba4caf43499a37f348abb98a66_0/explore

## Important to note that this includes Driving/Traffic citations.  Pittsburgh dataset does not.  

In [4]:
AsheCits #6494788 rows

,OBJECTID,citation_date,citation_time,address,citation_type,subject_race,subject_gender,agency,tcmainid,objectid_1
0,11659052,2018/06/04,1530,US 25/MARCELLUS ST,OPEN CONTAINER ALCOHOL VIOL,W,M,APD,258536,176303
1,11659053,2018/06/04,1530,US 25/MARCELLUS ST,SECOND DEGREE TRESPASS,W,M,APD,258536,176304
2,11659054,2018/06/14,1033,100-BLK SW PACK SQ,OPEN CONTAINER ALCOHOL VIOL,W,M,APD,258888,176305
3,11659055,2018/07/26,1930,126 BROADWAY ST,SECOND DEGREE TRESPASS,W,M,APD,259949,176306
4,11659056,2018/08/25,1843,US 25/240 E,SOLICIT ALMS/BEG FOR MONEY,W,M,APD,260857,176307
...,...,...,...,...,...,...,...,...,...,...
6494783,18153835,2020/11/26,1435,1400-BLK PATTON AVE/DEAVERVIEW RD,FAIL TO STOP-STEADY RED LIGHT,W,M,APD,291013,198620
6494784,18153836,2020/11/26,1435,1400-BLK PATTON AVE/DEAVERVIEW RD,NO OPERATORS LICENSE,W,M,APD,291013,198621
6494785,18153837,2020/11/01,1800,I 40/SMOKEY PARK HWY,SPEEDING,W,F,APD,291191,198622
6494786,18153838,2020/11/04,2017,200-BLK HAYWOOD RD,OPERATE VEH NO INS,W,M,APD,291275,198623


In [7]:
#convert times to datetimes and sort
AsheCits['citation_date'] = pd.to_datetime(AsheCits['citation_date'])
AsheCits = AsheCits.sort_values(['citation_date'], ascending=True)
#Date range covered: 2012-01-01 - 2020-11-30
AsheCits.tail()

,OBJECTID,citation_date,citation_time,address,citation_type,subject_race,subject_gender,agency,tcmainid,objectid_1
5229897,16888949,2020-11-30,1508,I 40/SMOKEY PARK HWY,SOLICITING FROM HIGHWAY,B,M,APD,290534,196889
4584593,16243645,2020-11-30,1551,ACTON CIRCLE/127 ACTON CIRCLE,MISDEMEANOR LARCENY,W,F,APD,290533,197115
6011898,17670950,2020-11-30,1928,I 240/MONTFORD ST EXIT,SPEEDING,NaN,M,APD,290536,198619
5931416,17590468,2020-11-30,1443,ACTON CIRCLE/ACTON WOODS RD,NO OPERATORS LICENSE,W,F,APD,290538,198618
5390646,17049698,2020-11-30,1649,I 240/I40W,SPEEDING,W,M,APD,290555,196701


In [8]:
#Periodical Cicadas are confirmed in Asheville for Brood VI (2017).  Reducing dataset to 2016-2018
AsheCitstrim = AsheCits[AsheCits['citation_date'] > dt.datetime(2016,5,1)]
AsheCitstrim = AsheCitstrim[AsheCitstrim['citation_date'] < dt.datetime(2018,7,1)]

In [9]:
AsheCits.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6494788 entries, 1479629 to 5390646
Data columns (total 10 columns):
 #   Column          Dtype         
---  ------          -----         
 0   OBJECTID        int64         
 1   citation_date   datetime64[ns]
 2   citation_time   int64         
 3   address         object        
 4   citation_type   object        
 5   subject_race    object        
 6   subject_gender  object        
 7   agency          object        
 8   tcmainid        int64         
 9   objectid_1      int64         
dtypes: datetime64[ns](1), int64(4), object(5)
memory usage: 545.1+ MB


## Getting the relevant years

In [9]:
#2016
#154981 rows!?! Seems like a lot
AsheCits2016 = AsheCitstrim[AsheCitstrim['citation_date'] > dt.datetime(2016,5,1)]
AsheCits2016 = AsheCitstrim[AsheCitstrim['citation_date'] < dt.datetime(2016,7,1)]

In [10]:
#2017
#926666 rows!?! Holy cow
AsheCits2017 = AsheCitstrim[AsheCitstrim['citation_date'] > dt.datetime(2017,5,1)]
AsheCits2017 = AsheCitstrim[AsheCitstrim['citation_date'] < dt.datetime(2017,7,1)]

In [11]:
#2018
#1521834 rows!
AsheCits2018 = AsheCitstrim[AsheCitstrim['citation_date'] > dt.datetime(2018,5,1)]
AsheCits2018 = AsheCitstrim[AsheCitstrim['citation_date'] < dt.datetime(2018,7,1)]

In [12]:
#Bring into a single dataset
years = [AsheCits2016,AsheCits2017,AsheCits2018]
AsheCits_final = pd.concat(years)

## What else can we learn about the data?

In [18]:
#what kinds of citations are there?
AsheCits_final['citation_type'].unique()

array(['NO OPERATORS LICENSE                                        ',
       'FOLLOWING TOO CLOSELY                                       ',
       'SPEEDING                                                    ',
       'DWLR NOT IMPAIRED REV                                       ',
       'EXPIRED REGISTRATION CARD/TAG                               ',
       'EXPIRED/NO INSPECTION                                       ',
       'SECOND DEGREE TRESPASS                                      ',
       'POSSESS DRUG PARAPHERNALIA                                  ',
       'FAIL MAINTAIN LANE CONTROL                                  ',
       'OPEN CONTAINER ALCOHOL VIOL                                 ',
       'OPERATE VEH NO INS                                          ',
       'CANCL/REVOK/SUSP CERTIF/TAG                                 ',
       'AID & ABET LARCENY (M)                                      ',
       'FAIL TO REPORT ACCIDENT                                     ',
      

## Export

In [13]:
#This is actually still a huge amount of data.  Going to trim off some columns
#data = data.drop(columns="cases")
AsheCits_final = AsheCits_final.drop(columns='address')

In [14]:
AsheCits_final = AsheCits_final.drop(columns='tcmainid') #not sure what this even is / not useful

In [15]:
AsheCits_final = AsheCits_final.drop(columns='objectid_1')

In [16]:
AsheCits_final = AsheCits_final.drop(columns='agency') #Asheville PD

In [17]:
# time of day seems out of scope for this investigation but could be revisited.  
#For the record, Periodical cicadas appear to typically sing between 10am and 5pm.
AsheCits_final = AsheCits_final.drop(columns='citation_time')

In [18]:
#STILL too much data! trimming more columns
AsheCits_final = AsheCits_final.drop(columns=['OBJECTID', 'subject_race'])

In [31]:
AsheCits_final.head()

,citation_date,citation_type,subject_gender
1601774,2016-05-02,NO OPERATORS LICENSE ...,F
399400,2016-05-02,FOLLOWING TOO CLOSELY ...,M
193291,2016-05-02,SPEEDING ...,M
1120542,2016-05-02,DWLR NOT IMPAIRED REV ...,M
2158530,2016-05-02,EXPIRED REGISTRATION CARD/TAG ...,M


In [32]:
#Save as a csv
AsheCits_final.to_csv('AsheCits_final.csv', index = False, header=True)

## Dataset is too large even after trimming almost everything.  I think I should move the traffic data to a separate df

In [71]:
List_AsheTrafficCits = ['FOLLOWING TOO CLOSELY','DRIVE AFTER CONSUMING < 21',
       'SPEEDING', 'NO OPERATORS LICENSE                                        ',
        'EXPIRED REGISTRATION CARD/TAG','HIT AND RUN - FAIL INFO',
       'FAIL SURR TITLE/REG CARD/TAG',
       'DRIVE WRONG WAY-ONE WAY DOT', 'EXCEEDING POSTED SPEED', 'FAIL STOP STOPSIGN/FLSH RED LT',
        'FAIL MAINTAIN LANE CONTROL',
      'OPERATE VEH NO INS','UNSEALED WINE/LIQ IN PASS AREA','LEARNERS PERMIT VIOLATION >18',
     'FAIL TO REPORT ACCIDENT','MOTORCYCLE FAIL BURN TAILLIGHT',
       'FAILURE TO REDUCE SPEED','IMPROPER LOADING/COVERING VEH',
       'EXCEEDING SAFE SPEED',
       'FAIL STOP STOPSIGN/FLSH RED LT','SPEED IN EXCESS OF 45 MPH',
       'SPEED IN EXCESS OF 65 MPH',
       'RECKLESS DRVG-WANTON DISREGARD',
       'FAIL TO WEAR SEAT BELT-DRIVER',
        'FAIL TO STOP-STEADY RED LIGH',
       'FAILURE TO YIELD',     
       'DRIVE WRONG WAY-ONE WAY ST/RD',
       'UNSAFE MOVEMENT','OPERATE COMM VEH COMM NOL',
       'HIT/RUN LEAVE SCENE PROP DAM',
        'UNSAFE MOVE MC/BC CHNG/LV LANE','MIRROR VIOLATION',
       'SPEED WORK ZN<=80AND<=15 OVER',
       'DRIVE LEFT OF CENTER','SPEED IN EXCESS OF 35 MPH','FICT/ALT TITLE/REG CARD/TAG',
        'SPEED COMPETITION','DRIVE LEFT OF CENTER-PASS RGHT',
       'FAIL TO YIELD FROM PRIVATE DRV',
       'UNSAFE LANE CHANGE','DRIVE WITHOUT TWO HEADLAMPS',
       'TRAFFIC INFRACTION - FREE TEXT',
       'UNSAFE PASSING YELLOW LINE ',
       'DRIVE/ALLOW MV NO REGISTRATION',
        'FAIL TO OBEY TRAFFIC OFFICER',
       'HIT/RUN FAIL STOP PROP DAMAGE',
        'DRIVE WITHOUT TWO HEADLAMPS','MOBILE PHONE VIOL MOTOR VEH',
        'LICENSE PLATE FRAME/COVER VIOL',
         'MOTORCYCLE/MOPED HELMET/PSGR',
        'FICT/ALT TITLE/REG CARD/TAG',
       'FAIL WEAR SEAT BELT-FRONT SEAT','DWLR VIOL LIMITED DRIVE PRIV',
       'DR/ALLOW REG PLATE NOT DISPLAY ',
       'NO LIABILITY INSURANCE','MOTORCYCLE FAIL BURN HEADLAMP',
       'FAIL TO YIELD LEFT TURN',
       'FAILURE TO DIM LIGHTS MEET VEH',
       'FAIL TO NOTIFY DMV ADDR CHANGE',
        'FAIL WEAR SEAT BELT-REAR SEAT',
       'EXCEEDING POSTED SPEED', 'EXPIRED OPERATORS LICENSE                                   ',
       'DIRECTIONAL SIGNALS EQUIP VIOL',
       'BRAKE/STOP LIGHT EQUIP VIOL', 'FICT/CNCL/REV REG CARD/TAG',
       'RECKLESS DRIVING TO ENDANGER','IMPROPER TURN',
        'TEXTING VIOL MOTOR VEH',
       'IMPROPER PASSING ON RIGHT',
       'FAIL PEDESTRIAN RIGHT OF WAY',
        'FAIL TO BURN HEADLAMPS',
       'ILLEGAL RIGHT TURN ON RED','LOW-SPEED VEHICLE VIOLATION',
       'UNSAFE MOVE MC/BC CRASH SBINJ',
       'DRIVE CVEH CLIC DISQUALIFIED','UNAUTHORIZED USE OF MOTOR VEH',
       'OVER LOAD SIZE/LENGTH/ VEHICLE',
       'WINDOW TINTING VIOL',
       'FAIL TO HEED LIGHT OR SIREN','HIT/RUN FAIL STOP PROP DAMAGE                               ',
       'FAILURE TO REDUCE SPEED'
       'UNSAFE TIRES','COVER REG STICKER/PLATE',
       'RECKLESS DRVG-WANTON DISREGARD',
        'HIT/RUN LEAVE SCENE PROP DAM','SPEEDING', 'NO LIABILITY INSURANCE',
       'REAR LAMPS VIOLATION', 'EXCEEDING SAFE SPEED',
       'DRIVING WHILE IMPAIRED', 'AGGRESSIVE DRIVING',
       'EXPIRED REGISTRATION CARD/TAG', 'OPERATE VEH NO INS',
       'FAIL CARRY REGISTRATION CARD', 'DRIVE/ALLOW MV NO REGISTRATION',
    'OVERLOADED/OVERCROWDED VEHICLE',
       'DRIVE WRONG WAY-ONE WAY LOCAL',    
      'LOCATION OF TV IN VEHICLE',
      'FAIL YLD STOPSIGN/FLSH RED LGT',
       'PERMIT OPERATION VEH NO INS','RECKLESS DRIVING TO ENDANGER',
       'UNSAFE LANE CHANGE', 'FAIL TO STOP FOR STOPPED BUS',
       'FAIL TO YIELD TO PEDESTRIAN',
       'AGGRESSIVE DRIVING',
       'UNLAWFULLY PASS EM/PUB SV VEH',
       'DRIVING WHILE IMPAIRED','DWLR NOT IMPAIRED REV                                       ',
       'FAIL EXHIBIT/SURRENDER LICENSE',             
         'NO MOTORCYCLE ENDORSEMENT',    
        'CENTER LANE VIOLATION',
       'SPEED WORK ZN >80 OR>15 OVER',
       'TRAFFIC INFRACTION - FREE TEXT',
       'SPEED IN EXCESS OF 55 MPH',
        'IMPROPER EQUIP - SPEEDOMETER',
        'REAR LAMPS VIOLATION',
        'RECKLESS DRVG W CMV LOAD PRMT',
        'FAIL CARRY VALID DRIVERS LIC',
       'STOP WHERE TRAFFIC OBSTRUCTED',           
       'WHITE LIGHT REAR-DRIVE FORWARD',
       'TRAFFIC OFFENSE - FREE TEXT',
       'TEXTING/EMAIL VIOL MOTOR VEH',
        'NO HEADLIGHTS ON WIPERS ON',
        'ALLOW UNLICENSED TO DRIVE',
        'IMPEDE TRAFFIC BY SLOW SPEED',
        'FAIL TO STOP FOR STOPPED BUS',
       'IMPROPER BRAKES',
        'IMPROPER PASSING',
        'FAIL TO YIELD TO PEDESTRIAN',
        'FAIL TO YIELD EMERG VEH NO ACC',
       'UNSAFE PASSING ONCOMING TRAF',
        'SPEEDING IN SCHOOL ZONE',
        'IMPROPER MUFFLER',
       'CHILD NOT IN REAR SEAT',
       'ADDITIONAL LIGHTING EQUIP VIOL',
                        'HIT/RUN FAIL STOP PROP DAMAGE'
        'IMPROPER VEHICLE INSPECTION',
       'NO REGISTRATION CARD',
       'LIC/PERMIT NO SUPV DRIVER <18',
       'LIC/PERMIT SEATING VIOL <18','DWLR NOT IMPAIRED REV'
       'REG PLAT IMPROPERLY ATTACHED', 
        'FAILURE TO YIELD - YIELD SIGN',
         'WRONG WAY ON DUAL LANE HIGHWAY',     
        'COVERING/DISGUISING REG PLATE',
       'IMPROPER STEERING MECHANISM',
       'USE OF RED OR BLUE LIGHT',
       'GIVE/LEND/BORROW LIC PLATE', 'DWLR',
       'NO OPERATORS LICENSE                                        ',
       'FOLLOWING TOO CLOSELY                                       ',
       'SPEEDING                                                    ',
       'EXPIRED REGISTRATION CARD/TAG                               ',
       'DWLR                                                        ',
       'DWLR IMPAIRED REV                                           ',
       'FAIL MAINTAIN LANE CONTROL                                  ',
       'CENTER LANE VIOLATION                                       ',
       'SPEED WORK ZN >80 OR>15 OVER                                ',
       'TRAFFIC INFRACTION - FREE TEXT                              ',
       'SPEED IN EXCESS OF 55 MPH                                   ',
       'RECKLESS DRVG W CMV LOAD PRMT                               ',
       'WHITE LIGHT REAR-DRIVE FORWARD                              ',
       'TRAFFIC OFFENSE - FREE TEXT                                 ',
       'TEXTING/EMAIL VIOL MOTOR VEH                                ',
       'NO HEADLIGHTS ON WIPERS ON                                  ',
       'DRIVE WITHOUT TWO HEADLAMPS                                 ',
       'FAIL TO STOP FOR STOPPED BUS                                ',
    'HIT/RUN LEAVE SCENE PROP DAM                                ',
    'MOBILE PHONE VIOL MOTOR VE                                  ',
    'EXCEEDING SAFE SPEED                                        ',
    'MOTORCYCLE FAIL BURN TAILLIGHT                              ',
    'FAIL STOP STOPSIGN/FLSH RED LT                               ',
    'FAIL TO HEED LIGHT OR SIREN                                 ',
                        'FAILURE TO REDUCE SPEED                                     ',
                        'FAIL STOP STOPSIGN/FLSH RED LT                              ',
                        'FAIL TO WEAR SEAT BELT-DRIVER                               ',
       'RECKLESS DRVG-WANTON DISREGARD                              ',
                        'FAIL TO STOP-STEADY RED LIGHT                               ',
       'FAILURE TO YIELD                                            ',
                        'DRIVE WRONG WAY-ONE WAY ST/RD                               ',
                        'UNSAFE MOVEMENT                                             ',
       'UNSAFE MOVE MC/BC CHNG/LV LANE                              ',
       'DRIVE LEFT OF CENTER                                        ',
                        'UNSAFE LANE CHANGE                                          ',
       'UNSAFE PASSING YELLOW LINE                                  ',
       'SPEED COMPETITION                                           ',
       'DRIVE/ALLOW MV NO REGISTRATION                              ',
                        'OPERATE VEH NO INS                                          ',
       'FAIL TO REPORT ACCIDENT                                     ',
                        'FAIL TO OBEY TRAFFIC OFFICER                                ',
                        'MOTORCYCLE/MOPED HELMET/PSGR                                ',
       'FAIL TO SECURE PASSEN UNDER 16                              ',
                        'IMPROPER BACKING                                            ',
       'FICT/ALT TITLE/REG CARD/TAG                                 ',
       'FAIL WEAR SEAT BELT-FRONT SEAT                              ',
       'DR/ALLOW REG PLATE NOT DISPLAY                              ',
       'NO LIABILITY INSURANCE                                      ',
                        'FAILURE TO DIM LIGHTS MEET VEH                              ',
       'FAIL TO NOTIFY DMV ADDR CHANGE                              ',
                        'FAIL WEAR SEAT BELT-REAR SEAT                               ',
                        'TEXTING VIOL MOTOR VEH                                      ',
       'IMPROPER PASSING ON RIGHT                                   ',
                        'FAIL TO BURN HEADLAMPS                                      ',
       'ILLEGAL RIGHT TURN ON RED                                   ',
       'UNSAFE TIRES                                                ',
                        'DRIVE WRONG WAY-ONE WAY LOCAL                               ',
                        'OVERLOADED/OVERCROWDED VEHICLE                              ',
                        'AGGRESSIVE DRIVING                                          ',
       'UNLAWFULLY PASS EM/PUB SV VEH                               ',
       'DRIVING WHILE IMPAIRED                                      ',
       'FAIL EXHIBIT/SURRENDER LICENSE                              ',
                        'NO MOTORCYCLE ENDORSEMENT                                   ',
                        'STOP WHERE TRAFFIC OBSTRUCTED                               ',
       'FAIL CARRY VALID DRIVERS LIC                                ',
                        'IMPROPER BRAKES                                             ',
                        'IMPROPER PASSING                                            ',
                        'IMPROPER MUFFLER                                            ',
       'CHILD NOT IN REAR SEAT                                      ',
       'ADDITIONAL LIGHTING EQUIP VIOL                              ',
                        'FAILURE TO YIELD - YIELD SIGN                               ',
                        'LEARNERS PERMIT VIOLATION >18                               ',
                        'UNSAFE MOVE MC/BC CRSH DMG/INJ                              ',
                        'HIT AND RUN - FAIL INFO                                     ',
       'FAIL SURR TITLE/REG CARD/TAG                                ',
                        'IMPROPER USE DEALER PLATES                                  ',
       'PASSENGER FLEE ACC PROP DAM                                 ',
       'IMPROPER LOADING/COVERING VEH                               ',
       'ILLEGAL PARKING                                             ',
                        'PARK IN UNDESIGNATED AREAS                                  ',
       'DRIVE LEFT OF CENTER-PASS RGHT                              ',
                        'OPERATE COMM VEH COMM NOL                                   ',
       'PARK FIRE HYD/STATION/PRIV DR                               ',
       'STOP/LEAVE STAND MV INTER/CAH                               ',
       'LEAVE VEH UNATTENDED/UNSECURE                               ',
       'MIRROR VIOLATION                                            ',
       'SPEED WORK ZN<=80AND<=15 OVER                               ',
       'OBSTRUCT PEDESTRIAN SIDEWALK                                ',
       'MOTORCYCLE FAIL BURN HEADLAMP                               ',
       'UNINTENTIONAL LITTER <= 15 LB                               ',
       'NO LIGHTS ON PARKED VEHICLE                                 ',
       'LOW-SPEED VEHICLE VIOLATION                                 ',
       'UNSAFE MOVE MC/BC CRASH SBINJ                               ',
                        'EXPIRED/NO INSPECTION                                       ',
                        'CANCL/REVOK/SUSP CERTIF/TAG                                 ',
                        'LICENSE PLATE FRAME/COVER VIOL                              ',
                        'FAIL TO YIELD LEFT TURN                                     ',
                        'BRAKE/STOP LIGHT EQUIP VIOL                                 ',
       'EXCEEDING POSTED SPEED                                      ',
       'DIRECTIONAL SIGNALS EQUIP VIOL                              ',
       'RECKLESS DRIVING TO ENDANGER                                ',
                        'IMPROPER EQUIP - SPEEDOMETER                                ',
              'ALLOW UNLICENSED TO DRIVE                                   ',
'IMPEDE TRAFFIC BY SLOW SPEED                                ',
                        'FICT/CNCL/REV REG CARD/TAG                                  ',
                        'IMPROPER TURN                                               ',
                        'FAIL TO YIELD EMERG VEH NO ACC                              ',
       'UNSAFE PASSING ONCOMING TRAF                                ',
                        'SPEEDING IN SCHOOL ZONE                                     ',
       'DRIVE CVEH CLIC DISQUALIFIED                                ',
                        'UNAUTHORIZED USE OF MOTOR VEH                               ',
                         'IMPROPER VEHICLE INSPECTION                                 ',
       'NO REGISTRATION CARD                                        ',
                        'LIC/PERMIT NO SUPV DRIVER <18                               ',
                        'WRONG WAY ON DUAL LANE HIGHWAY                              ',
       'LIC/PERMIT SEATING VIOL <18                                 ',
       'REG PLAT IMPROPERLY ATTACHED                                ',
       'OVER LOAD SIZE/LENGTH/ VEHICLE                              ',
       'WINDOW TINTING VIOL                                         ', 'ILLEGAL PARKING','OBSTRUCT PEDESTRIAN SIDEWALK',
                        'UNSAFE PASSING RR OR INTERSECT','UNSAFE TIRES', 'PARK FIRE HYD/STATION/PRIV DR',
                        'IMPROPER STEERING MECHANISM                                 ',
       'USE OF RED OR BLUE LIGHT                                    ',
       'GIVE/LEND/BORROW LIC PLATE                                  ',
       'UNSAFE MOVE MC/BC CRASH>5,000                               ',
                        'SPEED IN EXCESS OF 35 MPH                                   ',
       'HANDICAPPED PARKING VIOLATION                               ',
                        'SPEED IN EXCESS OF 45 MPH                                   ',
                        'FAIL TO YIELD TO PEDESTRIAN                                 ',
       'SPEED IN EXCESS OF 65 MPH                                   ',
                        'COVER REG STICKER/PLATE                                     ',
                        'UNSEALED WINE/LIQ IN PASS AREA                              ',
                        'SPEED IN EXCESS OF 35 MPH                                   ',
                        'REG CARD ADDRESS CHANGE VIO                                 ',
                        'DRIVE WRONG WAY ON DUAL LANE                                ',
                        'FAIL TO YIELD FROM PRIVATE DRV                              ',
                        'FAIL YLD STOPSIGN/FLSH RED LGT                              ',
       'PERMIT OPERATION VEH NO INS                                 ',
                        'DRIVE AFTER CONSUMING < 21                                  ','DR/ALLOW REG PLATE NOT DISPLAY',
                        'FAIL TO APPLY FOR NEW TITLE                                 ','UNSAFE MOVE MC/BC CRASH>5,000','DRINK BEER/WINE WHILE DRIVING',
                        'IMPROPER BACKING',
       'DRIVE WRONG WAY-ONE WAY DOT                                 ',
                        'MOBILE PHONE VIOL MOTOR VEH                                 ','FAIL TO SECURE PASSEN UNDER 16',
       'LIC/PRMT VIOL OTH RSTRCTN <18                               ',
                        'FAIL TO STOP-STEADY RED LIGHT',
                        'FAIL TO SECURE PASSEN UNDER 16','HANDICAPPED PARKING VIOLATION',
                        'FAIL PEDESTRIAN RIGHT OF WAY                                ',
                        'DWLR VIOL LIMITED DRIVE PRIV                                ', 'UNSAFE PASSING YELLOW LINE',
                        'UNSAFE MOVE MC/BC CRSH DMG/INJ', 'RECKLESS DRIVING AFT ALC(R/83)','IMPROPER VEHICLE INSPECTION', 'PARK IN UNDESIGNATED AREAS'
           ]

In [34]:
#Make new dataframe for just traffic citations
Ashe_Traffic = AsheCits_final[(AsheCits_final['citation_type'].isin(List_AsheTrafficCits))]

In [ ]:
Ashe_Traffic.info #1026972 rows so far

In [72]:
AsheCits_notTraffic = AsheCits_final[(~AsheCits_final['citation_type'].isin(List_AsheTrafficCits))]

In [73]:
AsheCits_notTraffic.info() #848040 rows

<class 'pandas.core.frame.DataFrame'>
Int64Index: 848040 entries, 2478669 to 2818664
Data columns (total 3 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   citation_date   848040 non-null  datetime64[ns]
 1   citation_type   848040 non-null  object        
 2   subject_gender  847466 non-null  object        
dtypes: datetime64[ns](1), object(2)
memory usage: 25.9+ MB


In [74]:
AsheCits_notTraffic2 = AsheCits_final[(~AsheCits_final['citation_type'].isin(List_AsheTrafficCits))]
AsheCits_notTraffic2.info() #848040 rows

<class 'pandas.core.frame.DataFrame'>
Int64Index: 848040 entries, 2478669 to 2818664
Data columns (total 3 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   citation_date   848040 non-null  datetime64[ns]
 1   citation_type   848040 non-null  object        
 2   subject_gender  847466 non-null  object        
dtypes: datetime64[ns](1), object(2)
memory usage: 25.9+ MB


In [57]:
AsheCits_notTraffic2['citation_type'].unique()

array(['SECOND DEGREE TRESPASS                                      ',
       'POSSESS DRUG PARAPHERNALIA                                  ',
       'OPEN CONTAINER ALCOHOL VIOL                                 ',
       'AID & ABET LARCENY (M)                                      ',
       'URINATE IN PUBLIC                                           ',
       'INFRACTION - FREE TEXT                                      ',
       'LITTERING NOT > 15 LBS                                      ',
       'COUNTY DOG ORDINANCE                                        ',
       'LOCAL ORDINANCE-FREE TEXT                                   ',
       'SOLICIT ALMS/BEG FOR MONEY                                  ',
       'SIMPLE POSSESS SCH VI CS (M)                                ',
       'MISDEMEANOR LARCENY                                         ',
       'POSS OPN CNT/CONS ALC PSG AREA                              ',
       'EXPIRED OPERATORS LICENSE                                   ',
      

In [75]:
#How big is this file now?
AsheCits_final2 = AsheCits_notTraffic2

#export
AsheCits_final2.to_csv('AsheCits_final2.csv', index = False, header=True)